# Statistics

In [1]:
import pandas as pd
import json
import random

In [3]:
with open('../raw/FN_11934_filtered.json', 'r') as f:
    FN = json.load(f)
    
with open('../raw/DN_27505_filtered.json', 'r') as f:
    DN = json.load(f)
    
len(FN), len(DN)

(11934, 27505)

## Matching Pairs (Total)

In [4]:
matching_num = [len(fn['debunking_ids']) for fn in FN]
df = pd.DataFrame({'num': matching_num})
len(df)

11934

In [5]:
df.describe()

,num
count,11934.000000
mean,7.237473
std,10.306454
min,1.000000
25%,1.000000
50%,3.000000
75%,8.000000
max,138.000000


In [6]:
df[df['num']>25].describe()

,num
count,851.000000
mean,37.296122
std,13.728090
min,26.000000
25%,30.000000
50%,36.000000
75%,38.000000
max,138.000000


In [7]:
sum(matching_num)

86372

## Sentences Num of DN

In [8]:
sents_num = [len(dn['content_all']) for dn in DN]
df = pd.DataFrame({'num':sents_num})
len(df)

27505

In [9]:
df.describe()

,num
count,27505.000000
mean,15.208653
std,20.060215
min,1.000000
25%,4.000000
50%,7.000000
75%,19.000000
max,356.000000


In [10]:
df[df['num']<=5].describe()

,num
count,10942.000000
mean,3.216871
std,1.330441
min,1.000000
25%,2.000000
50%,3.000000
75%,4.000000
max,5.000000


In [11]:
df[df['num']<=3].describe()

,num
count,6010.000000
mean,2.176539
std,0.785084
min,1.000000
25%,2.000000
50%,2.000000
75%,3.000000
max,3.000000


In [12]:
len(df[df['num'] <= 3]) / len(df), len(df[df['num'] <= 5]) / len(df)

(0.21850572623159426, 0.39781857844028357)

# Splits

## Loading

In [1]:
import pandas as pd
import json
import random
import numpy as np
from tqdm import tqdm

In [2]:
with open('../raw/FN_11934_filtered.json', 'r') as f:
    FN = json.load(f)

with open('../raw/DN_27505_filtered.json', 'r') as f:
    DN = json.load(f)

len(FN), len(DN)

(11934, 27505)

In [3]:
fnOid2item = {fn['_id']: fn for fn in FN}
dnOid2item = {dn['_id']: dn for dn in DN}

fnOid2idx = {fn['_id']: i for i, fn in enumerate(FN)}
dnOid2idx = {dn['_id']: i for i, dn in enumerate(DN)}

In [4]:
dnOid2sentsLen = {dn['_id']:len(dn['content_all']) for dn in DN}

In [5]:
list(dnOid2sentsLen.items())[:5]

[('5fef066911f159c49f84b0e8', 28),
 ('5fed832711f159c49f84b0e3', 189),
 ('5f64bf8c11f159c49f84af7e', 6),
 ('5f64bf8b11f159c49f84af79', 6),
 ('5f64bf8c11f159c49f84af7b', 6)]

In [6]:
FN[0]

{'_id': '5f0ed06ada7c229f16a54452',
 'content': '再发一条简体中文的：谢才萍，江湖上人称“谢姐”，是重庆目前已逮捕的十九个黑恶团伙首犯中唯一一位女性，江湖地位甚高。谢才萍与文强是“亲戚关系”：她是文强的弟媳。让人瞠目的是，谢才萍在个人生活上极度荒淫。重庆市打黑成果展显示，她长期包养十六个年轻男子供自己玩乐。',
 'event': '5f5e37e063fc4d00ff6bedcc',
 'time': 1255276800000,
 'time_format': '2009-10-12 00:00:00',
 'debunking_ids': ['6002d4fbbc9086321270c5dc'],
 'content_all': '再发一条简体中文的：谢才萍，江湖上人称“谢姐”，是重庆目前已逮捕的十九个黑恶团伙首犯中唯一一位女性，江湖地位甚高。谢才萍与文强是“亲戚关系”：她是文强的弟媳。让人瞠目的是，谢才萍在个人生活上极度荒淫。重庆市打黑成果展显示，她长期包养十六个年轻男子供自己玩乐。'}

In [7]:
def get_qids(event_file):
    with open(event_file, 'r') as f:
        events = json.load(f)
        
    qids = []
    for fn in FN:
        if fn['event'] in events:
            qids.append(fn['_id'])
            
    print(len(qids), len(set(qids)))
    return qids

In [9]:
train_qids, val_qids, test_qids = get_qids('./data/data_splits_setting1/events.train'), get_qids(
    './data/data_splits_setting1/events.dev'), get_qids('./data/data_splits_setting1/events.test')

8356 8356
1192 1192
2386 2386


In [10]:
len(train_qids) + len(val_qids) + len(test_qids)

11934

In [11]:
a = set(train_qids)
b = set(val_qids)
c = set(test_qids)

a.intersection(b), a.intersection(c), b.intersection(c), len(a), len(b), len(c)

(set(), set(), set(), 8356, 1192, 2386)

In [12]:
len(test_qids) / len(FN), len(val_qids) / len(FN)

(0.199932964638847, 0.09988268811798223)

In [ ]:
bm25_arr = np.load('./data/bm25/bm25_scores_(11934, 27505).npy')
bm25_arr.shape

In [13]:
bm25_arr[0]

array([147.65676972, 175.41249475, 152.70300736, ..., 147.03297213,
       175.86724546, 150.45702979])

In [14]:
def analysis_qid2dids(qid2dids):
    is_pair = []
    for qid, dids in qid2dids.items():
        is_pair.append(len(set(dids).intersection(
            set(fnOid2item[qid]['debunking_ids']))))

    df = pd.DataFrame({'matching_num': is_pair})
    return df.describe()

In [15]:
def transfer2lineDf(qid2dids):
    Qids = []
    Qidxs = []
    Dids = []
    Didxs = []
    Labels = []

    for qid, dids in qid2dids.items():
        Qids += [qid] * len(dids)
        Qidxs += [fnOid2idx[qid]] * len(dids)
        Dids += dids
        Didxs += [dnOid2idx[did] for did in dids]
        Labels += [1 if did in fnOid2item[qid]
                   ['debunking_ids'] else 0 for did in dids]

    df_line = pd.DataFrame(dict(zip(['qid', 'qidx', 'did', 'didx', 'label'], [
                           Qids, Qidxs, Dids, Didxs, Labels])))

    return df_line


def transfer2df(qid2dids):
    Qids = []
    Qidxs = []
    Dids = []
    Didxs = []
    Labels = []

    for qid, dids in qid2dids.items():
        Qids += [qid]
        Qidxs += [fnOid2idx[qid]]
        Dids += [dids]
        Didxs += [[dnOid2idx[did] for did in dids]]
        Labels += [[1 if did in fnOid2item[qid]
                    ['debunking_ids'] else 0 for did in dids]]

    df = pd.DataFrame(dict(zip(['qid', 'qidx', 'did', 'didx', 'label'], [
                           Qids, Qidxs, Dids, Didxs, Labels])))

    return df

In [16]:
def analysis_lineDf(df):
    qids = set(df['qid'])
    dids = set(df['did'])

    total_pairs = sum([len(fnOid2item[qid]['debunking_ids']) for qid in qids])
    top50_pairs = len(df[df['label'] == 1])

    print('qids = {}, dids = {}, total pairs = {}, top50 pairs = {}'.format(
        len(qids), len(dids), total_pairs, top50_pairs))
    
    return df.describe()

## setting1: 尽可能多的使用正例（最多使用25个）

In [15]:
def extra_neg_dids(qid, num, existing_pos_dids):
    extra_dids = []

    score = bm25_arr[fnOid2idx[qid]]
    # choose bm25 top scores and not exitst in pos_dids
    for r in np.argsort(score)[::-1]:
        if len(extra_dids) == num:
            return extra_dids
        elif DN[r]['_id'] in existing_pos_dids:
            continue
        else:
            extra_dids.append(DN[r]['_id'])


def get_dids_from_qids(qids, TOP=50, MAX=25):
    qid2dids = dict()

    for qid in tqdm(qids):
        debunking_ids = fnOid2item[qid]['debunking_ids']

        if len(debunking_ids) <= MAX:
            chosen_dids = debunking_ids
        else:
            sents_num = [dnOid2sentsLen[did] for did in debunking_ids]
            sents_num_rank = np.argsort(sents_num)[::-1]
            chosen_dids = [debunking_ids[r] for r in sents_num_rank[:MAX]]

        dids = chosen_dids + \
            extra_neg_dids(qid, TOP - len(chosen_dids), debunking_ids)

        qid2dids[qid] = dids

    print(len(qid2dids), len(qid2dids)*50,
          sum([len(v) for v in qid2dids.values()]))
    print()

    return qid2dids

In [16]:
train_qid2dids = get_dids_from_qids(qids=train_qids)
val_qid2dids = get_dids_from_qids(qids=val_qids)
test_qid2dids = get_dids_from_qids(qids=test_qids)

  4%|▍         | 53/1192 [00:00<00:02, 529.81it/s]

8356 417800 417800



  2%|▏         | 40/2386 [00:00<00:05, 393.87it/s]

1192 59600 59600



100%|██████████| 2386/2386 [00:04<00:00, 495.35it/s]

2386 119300 119300



In [26]:
analysis_qid2dids(train_qid2dids)

,matching_num
count,8356.000000
mean,6.599090
std,7.370319
min,1.000000
25%,1.000000
50%,3.000000
75%,8.000000
max,25.000000


In [27]:
analysis_qid2dids(val_qid2dids)

,matching_num
count,1192.000000
mean,5.568792
std,5.593744
min,1.000000
25%,1.000000
50%,3.000000
75%,8.000000
max,25.000000


In [28]:
analysis_qid2dids(test_qid2dids)

,matching_num
count,2386.000000
mean,5.921207
std,6.634678
min,1.000000
25%,1.000000
50%,3.000000
75%,7.000000
max,25.000000


In [18]:
train_line_df = transfer2lineDf(train_qid2dids)
val_line_df = transfer2lineDf(val_qid2dids)
test_line_df = transfer2lineDf(test_qid2dids)

In [19]:
val_df = transfer2df(val_qid2dids)
test_df = transfer2df(test_qid2dids)

In [24]:
analysis_lineDf(train_line_df)

qids = 8356, dids = 17482, total pairs = 63873, top50 pairs = 55142


,qidx,didx,label
count,417800.000000,417800.00000,417800.000000
mean,6001.999282,12241.55505,0.131982
std,3466.800540,7278.74103,0.338471
min,1.000000,0.00000,0.000000
25%,2885.750000,6728.00000,0.000000
50%,6247.500000,11594.00000,0.000000
75%,8931.250000,17954.00000,0.000000
max,11933.000000,27504.00000,1.000000


In [29]:
analysis_lineDf(val_line_df)

qids = 1192, dids = 8306, total pairs = 7374, top50 pairs = 6638


,qidx,didx,label
count,59600.000000,59600.000000,59600.000000
mean,5544.696309,11782.188322,0.111376
std,3227.497234,7280.358121,0.314600
min,12.000000,1.000000,0.000000
25%,3145.750000,6125.000000,0.000000
50%,4886.000000,11157.000000,0.000000
75%,8388.250000,17181.000000,0.000000
max,11913.000000,27504.000000,1.000000


In [30]:
analysis_lineDf(test_line_df)

qids = 2386, dids = 11688, total pairs = 15125, top50 pairs = 14128


,qidx,didx,label
count,119300.000000,119300.000000,119300.000000
mean,6052.903185,12117.785054,0.118424
std,3459.273897,7294.357713,0.323111
min,0.000000,0.000000,0.000000
25%,3121.000000,6737.000000,0.000000
50%,5592.000000,11431.000000,0.000000
75%,9161.000000,17772.000000,0.000000
max,11928.000000,27502.000000,1.000000


In [33]:
analysis_lineDf(pd.concat([train_line_df, val_line_df, test_line_df]))

qids = 11934, dids = 19239, total pairs = 86372, top50 pairs = 75908


,qidx,didx,label
count,596700.000000,596700.000000,596700.000000
mean,5966.500000,12170.926564,0.127213
std,3445.051931,7283.330112,0.333212
min,0.000000,0.000000,0.000000
25%,2983.000000,6662.000000,0.000000
50%,5966.500000,11513.000000,0.000000
75%,8950.000000,17867.000000,0.000000
max,11933.000000,27504.000000,1.000000


In [34]:
train_line_df.to_csv('./data/data_splits/top50.train.line', index=None, header=None, sep='\t')
val_line_df.to_csv('./data/data_splits/top50.val.line', index=None, header=None, sep='\t')
test_line_df.to_csv('./data/data_splits/top50.test.line', index=None, header=None, sep='\t')

In [35]:
val_df.head()

,qid,qidx,did,didx,label
0,5f0ed06ada7c229f16a55c17,12,"[5f0c71f1efe4e78f5b003532, 5f0c71f1efe4e78f5b0...","[338, 309, 190, 7061, 27006, 16677, 14492, 245...","[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,5f0ed06ada7c229f16a55c1d,18,"[5f0c71f1efe4e78f5b003532, 5f0c71f1efe4e78f5b0...","[338, 309, 190, 1203, 20158, 14492, 27006, 18,...","[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,5f0ed06ada7c229f16a55c1c,24,"[5f0c71f1efe4e78f5b003532, 5f0c71f1efe4e78f5b0...","[338, 309, 190, 9233, 11057, 14701, 9057, 9229...","[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,5f0ed040da7c229f16a51432,51,"[5f61d4bd11c8070a282e796e, 5f61d4bd11c8070a282...","[66, 68, 69, 11046, 11385, 26092, 10768, 9571,...","[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,5f0ed06ada7c229f16a55c1b,148,"[5f0c71f1efe4e78f5b003532, 5f0c71f1efe4e78f5b0...","[338, 309, 190, 20500, 27261, 11388, 17329, 96...","[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [36]:
test_df.head()

,qid,qidx,did,didx,label
0,5f0ed06ada7c229f16a54452,0,"[6002d4fbbc9086321270c5dc, 5ff55038c60dfb55fc5...","[19, 21, 8931, 17396, 6883, 13539, 11103, 9365...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,5f0ed06ada7c229f16a53f06,2,"[5fe5acd511f159c49f84b00e, 5fe5acd511f159c49f8...","[24, 29, 25, 26, 27, 91, 22, 17, 18, 3180, 543...","[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,5f0ed06ada7c229f16a542cb,5,"[5fef252111f159c49f84b0f9, 5fef252111f159c49f8...","[37, 30, 31, 49, 19315, 20021, 184, 9837, 3180...","[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,5f0ed06ada7c229f16a55040,15,"[5f0c71f1efe4e78f5b002ef5, 5f2ea72f8d1bbb4d114...","[129, 17301, 11733, 2923, 17956, 19748, 14283,...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,5f0ed040da7c229f16a524f2,30,"[5f62cacffff4579430a77648, 5f1172acb9cce3edf31...","[64, 11139, 1257, 1245, 737, 1598, 8225, 440, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [37]:
val_df.to_csv('./data/data_splits/top50.val',
              index=None, header=None, sep='\t')
test_df.to_csv('./data/data_splits/top50.test',
               index=None, header=None, sep='\t')

## setting2: 尽可能遵循bm25的结果，最多添加1个正例

In [17]:
bm25_arr[0]

array([147.65676972, 175.41249475, 152.70300736, ..., 147.03297213,
       175.86724546, 150.45702979])

In [18]:
def get_dids_from_qids(qids, TOP=50):
    qid2dids = dict()

    for qid in tqdm(qids):
        # First: select top50 candidates
        scores = bm25_arr[fnOid2idx[qid]]
        scores_rank = scores.argsort()[::-1]
        candidates = [DN[r]['_id'] for r in scores_rank[:TOP]]
        
        debunking_ids = fnOid2item[qid]['debunking_ids']
        
        # top50 do not contain any positives
        if len(set(candidates).intersection(set(debunking_ids))) == 0:
            for r in scores_rank[TOP:]:
                did = DN[r]['_id']
                if did in debunking_ids:
                    candidates[-1] = did
                    # 注意：此处在acl投稿时忘记了加break...
#                     break

        qid2dids[qid] = candidates

    print(len(qid2dids), len(qid2dids)*50,
          sum([len(v) for v in qid2dids.values()]))
    print()

    return qid2dids

In [19]:
train_qid2dids = get_dids_from_qids(qids=train_qids)
val_qid2dids = get_dids_from_qids(qids=val_qids)
test_qid2dids = get_dids_from_qids(qids=test_qids)

  2%|▏         | 18/1192 [00:00<00:06, 173.16it/s]

8356 417800 417800



  1%|          | 24/2386 [00:00<00:10, 235.04it/s]

1192 59600 59600



100%|██████████| 2386/2386 [00:08<00:00, 267.88it/s]

2386 119300 119300



In [20]:
analysis_qid2dids(train_qid2dids)

,matching_num
count,8356.000000
mean,3.422212
std,4.464883
min,1.000000
25%,1.000000
50%,2.000000
75%,4.000000
max,50.000000


In [21]:
analysis_qid2dids(val_qid2dids)

,matching_num
count,1192.000000
mean,2.799497
std,3.271263
min,1.000000
25%,1.000000
50%,1.000000
75%,3.000000
max,18.000000


In [22]:
analysis_qid2dids(test_qid2dids)

,matching_num
count,2386.000000
mean,3.036463
std,3.568193
min,1.000000
25%,1.000000
50%,2.000000
75%,4.000000
max,32.000000


In [23]:
train_line_df = transfer2lineDf(train_qid2dids)
val_line_df = transfer2lineDf(val_qid2dids)
test_line_df = transfer2lineDf(test_qid2dids)

In [24]:
val_df = transfer2df(val_qid2dids)
test_df = transfer2df(test_qid2dids)

In [25]:
analysis_lineDf(train_line_df)

qids = 8356, dids = 17385, total pairs = 63873, top50 pairs = 28596


,qidx,didx,label
count,417800.000000,417800.000000,417800.000000
mean,6001.999282,12253.957738,0.068444
std,3466.800540,7264.769062,0.252507
min,1.000000,0.000000,0.000000
25%,2885.750000,6773.000000,0.000000
50%,6247.500000,11580.000000,0.000000
75%,8931.250000,17954.000000,0.000000
max,11933.000000,27504.000000,1.000000


In [26]:
analysis_lineDf(val_line_df)

qids = 1192, dids = 8353, total pairs = 7374, top50 pairs = 3337


,qidx,didx,label
count,59600.000000,59600.000000,59600.000000
mean,5544.696309,11975.893909,0.055990
std,3227.497234,7298.715065,0.229904
min,12.000000,1.000000,0.000000
25%,3145.750000,6330.000000,0.000000
50%,4886.000000,11355.000000,0.000000
75%,8388.250000,17418.000000,0.000000
max,11913.000000,27504.000000,1.000000


In [27]:
analysis_lineDf(test_line_df)

qids = 2386, dids = 11715, total pairs = 15125, top50 pairs = 7245


,qidx,didx,label
count,119300.000000,119300.000000,119300.000000
mean,6052.903185,12187.515331,0.060729
std,3459.273897,7329.480866,0.238834
min,0.000000,0.000000,0.000000
25%,3121.000000,6684.000000,0.000000
50%,5592.000000,11500.000000,0.000000
75%,9161.000000,17931.250000,0.000000
max,11928.000000,27503.000000,1.000000


In [28]:
analysis_lineDf(pd.concat([train_line_df, val_line_df, test_line_df]))

qids = 11934, dids = 19047, total pairs = 86372, top50 pairs = 39178


,qidx,didx,label
count,596700.000000,596700.000000,596700.000000
mean,5966.500000,12212.899948,0.065658
std,3445.051931,7281.607675,0.247683
min,0.000000,0.000000,0.000000
25%,2983.000000,6684.000000,0.000000
50%,5966.500000,11536.000000,0.000000
75%,8950.000000,17940.000000,0.000000
max,11933.000000,27504.000000,1.000000


In [29]:
train_line_df.to_csv('./data/data_splits_setting2/top50.train.line',
                     index=None, header=None, sep='\t')
val_line_df.to_csv('./data/data_splits_setting2/top50.val.line',
                   index=None, header=None, sep='\t')
test_line_df.to_csv('./data/data_splits_setting2/top50.test.line',
                    index=None, header=None, sep='\t')

In [30]:
val_df.head()

,qid,qidx,did,didx,label
0,5f0ed06ada7c229f16a55c17,12,"[5ff5e362c60dfb55fc52b972, 5ff967deefd97ff86d6...","[7061, 27006, 16677, 14492, 24503, 19881, 453,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,5f0ed06ada7c229f16a55c1d,18,"[5ff57ff6c60dfb55fc52b944, 5f2ea72f8d1bbb4d114...","[1203, 20158, 14492, 27006, 18, 27261, 27339, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,5f0ed06ada7c229f16a55c1c,24,"[5f1172acb9cce3edf31425da, 5f1172acb9cce3edf31...","[9233, 11057, 14701, 9057, 9229, 20500, 1457, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,5f0ed040da7c229f16a51432,51,"[5f61d4bd11c8070a282e796e, 5f1172acb9cce3edf31...","[66, 11046, 11385, 26092, 10768, 68, 9571, 159...","[1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
4,5f0ed06ada7c229f16a55c1b,148,"[5f0c71f1efe4e78f5b003534, 5f315f59aaa0d4d489f...","[190, 20500, 27261, 11388, 17329, 9648, 15047,...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [31]:
test_df.head()

,qid,qidx,did,didx,label
0,5f0ed06ada7c229f16a54452,0,"[6002d4fbbc9086321270c5dc, 5ff55038c60dfb55fc5...","[19, 21, 8931, 17396, 6883, 13539, 11103, 9365...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,5f0ed06ada7c229f16a53f06,2,"[5fe5acd511f159c49f84b012, 5fe5acd511f159c49f8...","[91, 29, 27, 25, 26, 24, 22, 17, 18, 3180, 543...","[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,5f0ed06ada7c229f16a542cb,5,"[5fef252111f159c49f84b0fa, 5fef26f911f159c49f8...","[31, 49, 37, 19315, 20021, 184, 9837, 3180, 11...","[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,5f0ed06ada7c229f16a55040,15,"[5f2ea72f8d1bbb4d1141aa43, 5f1172acb9cce3edf31...","[17301, 11733, 2923, 17956, 19748, 14283, 1801...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,5f0ed040da7c229f16a524f2,30,"[5f62cacffff4579430a77648, 5f1172acb9cce3edf31...","[64, 11139, 1257, 1245, 737, 1598, 8225, 440, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [32]:
val_df.to_csv('./data/data_splits_setting2/top50.val',
              index=None, header=None, sep='\t')
test_df.to_csv('./data/data_splits_setting2/top50.test',
               index=None, header=None, sep='\t')